In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from bert_serving.client import BertClient

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_hdf('metadata/preprocessed.h5')
df.head()

,paper_id,body_text,methods,results,source,title,doi,abstract,publish_time,authors,journal,arxiv_id,url,publish_year,is_covid19,study_design
0,3cdc48bb9e40afd30a59463b7872761a726998c8,NDV (Roakin strain) was obtained from Dr. D. J...,NDV (Roakin strain) was obtained from Dr. D. J...,Adult house flies harbored Newcastle Disease v...,PMC,Experimental Evaluation of Musca domestica (Di...,10.1093/jmedent/44.4.666,"House flies, Musca domestica L. (Diptera: Musc...",2007-07-01,"Watson, D. Wes; Niño, Elina L.; Rochon, Katery...",J Med Entomol,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2007,False,[]
1,d99acb4e99be7852aa61a688c9fbd38d44b5a252,Live attenuated viruses have been developed an...,RSV A2 strain was obtained from ATCC (Manassas...,The reverse genetics system for measles Edmons...,PMC,Evaluation of Measles Vaccine Virus as a Vecto...,10.2174/1874357901206010012,Live attenuated recombinant measles vaccine vi...,2012-02-16,"Mok, Hoyin; Cheng, Xing; Xu, Qi; Zengel, James...",Open Virol J,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,2012,False,"['truncated', 'gamma', 'protocol']"
2,748d4c57fe1acc8d9d97cf574f7dea5296f9386c,Ebola virus (EBOV) and other members of the fa...,U2OS human osteosarcoma cells were cultured in...,For evaluating EBOV GP triggering under biosaf...,PMC,Direct Visualization of Ebola Virus Fusion Tri...,10.1128/mbio.01857-15,Ebola virus (EBOV) makes extensive and intrica...,2016-02-09,"Spence, Jennifer S.; Krause, Tyler B.; Mittler...",mBio,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2016,False,"['truncated', 'heterogeneity']"
3,b891efc6e1419713b05ff7d89b26d260478c28df,To the Editor:\nChina has the world's second l...,NaN,NaN,PMC,Tuberculosis prevention in healthcare workers ...,10.1183/23120541.00015-2015,BSL3 and respiratory isolation wards protect h...,2015-08-21,"Deng, Yunfeng; Li, Yan; Wang, Fengtian; Gao, D...",ERJ Open Res,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,2015,False,[]
4,353852971069ad5794445e5c1ab6077ce23da75d,Coronavirus disease 2019 (COVID-19) has spread...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,-1,True,[]


## Extract papers

In [5]:
bc = BertClient(ip='0.0.0.0')

In [6]:
notna = df['abstract'].dropna().index
vec = df.loc[notna, 'abstract'].progress_apply(lambda x: bc.encode([x]))

  0%|          | 0/41062 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)
100%|██████████| 41062/41062 [13:50<00:00, 49.43it/s]


In [18]:
v = np.stack(vec.values).squeeze()
v

array([[-2.65608966e-01, -1.93127170e-01, -2.88924515e-01, ...,
        -5.88914692e-01, -6.77560419e-02,  2.41620958e-01],
       [ 1.59004033e-01, -6.18593574e-01, -3.38294625e-01, ...,
        -2.14150906e-01, -3.34556326e-02,  5.61481761e-03],
       [-8.59429613e-02,  1.24388486e-01, -2.88748235e-01, ...,
        -6.36165679e-01, -3.02090466e-01,  5.49710453e-01],
       ...,
       [-4.98621851e-01, -1.64846629e-01, -2.82329291e-01, ...,
        -4.41766560e-01, -3.13125998e-01,  9.55489576e-02],
       [-2.22110465e-01,  9.58468318e-02, -4.01151702e-02, ...,
        -3.09445202e-01, -2.81928092e-01,  1.67688161e-01],
       [-5.74713409e-01, -2.96211238e-05, -1.75356343e-01, ...,
        -5.11865675e-01, -5.63501157e-02,  3.88442606e-01]], dtype=float32)

In [24]:
prefix = 'metadata/uncased_L-12_H-768_A-12-'

np.save(f'{prefix}vec.npy', v)
np.save(f'{prefix}notna_index.npy', notna.values)